In [ ]:
%autosave 60
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
from pathlib import Path

In [ ]:
p = Path("../").resolve()

In [ ]:
import json
import logging
import os
import pickle
import sys
from collections import Counter, OrderedDict
from copy import deepcopy
from datetime import datetime, timedelta
from io import BytesIO
from pathlib import Path
from types import ModuleType
from typing import Dict, List, Optional, Tuple, Union, cast

import cv2
import matplotlib as plt
import numpy as np
import pandas as pd
import PIL
import PIL.Image as pil_img
import seaborn as sns
import sklearn as skl
from IPython.display import Image, display
from matplotlib.patches import Rectangle
from matplotlib_inline.backend_inline import set_matplotlib_formats
from termcolor import colored
from tqdm.contrib import tenumerate, tmap, tzip
from tqdm.contrib.bells import tqdm, trange

from geoscreens.pseudolabels import reverse_point

In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 15)
pd.set_option("display.max_rows", 50)
# Suitable default display for floats
pd.options.display.float_format = "{:,.2f}".format
# matplotlib options
set_matplotlib_formats("pdf", "png")
plt.rcParams["savefig.dpi"] = 75
plt.rcParams["figure.autolayout"] = False
plt.rcParams["figure.figsize"] = 10, 6
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["font.size"] = 16
plt.rcParams["lines.linewidth"] = 2.0
plt.rcParams["lines.markersize"] = 8
plt.rcParams["legend.fontsize"] = 14
plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "cm"
plt.rcParams["text.latex.preamble"] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

# This one is optional -- change graphs to SVG only use if you don't have a
# lot of points/lines in your graphs. Can also just use ['retina'] if you
# don't want SVG.
%config InlineBackend.figure_formats = ["retina"]
set_matplotlib_formats("pdf", "png")

In [ ]:
# VIDEO_PATH = Path("/shared/g-luo/geoguessr/videos").resolve()
# OUT_PATH = Path("/shared/gbiamby/geo/screenshots/screen_samples_auto").resolve()
# assert VIDEO_PATH.exists()
# assert OUT_PATH.exists()

## Game State Detection

### Load Detections

In [ ]:
def parse_tuple(s: str):
    if isinstance(s, str):
        result = s.replace("(", "[").replace(")", "]")
        result = result.replace("'", '"').strip()
        result = result.replace(",]", "]")
        if result:
            # print(result)
            return tuple(sorted((json.loads(result))))
        else:
            return set()
    return s


def parse_dict(s: str):
    if isinstance(s, str):
        return json.loads(s.replace("'", '"'))
    return s


def load_detections_csv(
    video_id: str, split: str = "val", model: str = "geoscreens_009-resnest50_fpn-with_augs"
) -> pd.DataFrame:
    csv_path = Path(
        f"/shared/gbiamby/geo/segment/detections/{model}/{split}/df_frame_dets-video_id_{video_id}.csv",
    )
    df = pd.read_csv(csv_path)
    df.frame_id = df.frame_id.astype(int)
    df.label_ids = df.label_ids.apply(lambda x: parse_dict(x))
    df.labels = df.labels.apply(lambda x: parse_dict(x))
    df.labels_set = df.labels_set.apply(lambda x: parse_tuple(x))
    df.scores = df.scores.apply(lambda x: parse_dict(x))
    df.bboxes = df.bboxes.apply(lambda x: parse_dict(x))

    return df


def load_detections(
    video_id: str, split: str = "val", model: str = "geoscreens_009-resnest50_fpn-with_augs"
) -> pd.DataFrame:
    dets_path = Path(
        f"/shared/gbiamby/geo/segment/detections/{model}/{split}/df_frame_dets-video_id_{video_id}.csv",
    )
    if dets_path.suffix == ".csv":
        df = load_detections_csv(video_id, split=split, model=model)
    else:
        df = pickle.load(open(dets_path, "rb"))

    if "frame_time" not in df.columns:
        df["frame_time"] = df.apply(lambda x: f"{x.frame_id/4.0:04}", axis=1)
    if "seconds" not in df.columns:
        df["seconds"] = df.frame_id.apply(lambda frame_id: frame_id / 4.0)
    if "time" not in df.columns:
        df["time"] = df.frame_id.apply(
            lambda frame_id: datetime.utcfromtimestamp(frame_id / 4.0).strftime("%H:%M:%S:%f")
        )
    return df


# def get_categories(geoscreens_version: str = "009"):
#     data = json.load(
#         open(
#             f"/home/gbiamby/proj/geoscreens/datasets/geoscreens_{geoscreens_version}/geoscreens_{geoscreens_version}.json",
#             "r",
#         )
#     )
#     return data["categories"]

In [ ]:
# # Show most common sets of UI elements that appear together in a single frame
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(pd.DataFrame(df_framedets.labels_set.value_counts()))

### Detect Game States for Each Frame

In [ ]:
def apply_smoothing(
    df_framedets: pd.DataFrame, window_size: int = 5, direction: str = "forward"
) -> None:
    smoothed = []
    current_state = df_framedets.loc[0]["game_state"]
    direction = direction.replace("backwards", "backward").replace("forwards", "forward")
    prev_state = current_state
    if direction == "backward":
        buffer = [df_framedets.loc[0]["game_state"]] * window_size
        for i, row in df_framedets.iterrows():
            buffer.pop(0)
            buffer.append(row.game_state)
            counter = Counter(buffer)
            current_state = counter.most_common()[0]
            smoothed.append(current_state)
    else:
        buffer = [df_framedets.loc[0]["game_state"]] * window_size
        for i, row in df_framedets.iterrows():
            buffer.pop(0)
            buffer.append(
                df_framedets.loc[min(i + window_size, df_framedets.shape[0] - 1)].game_state
            )
            if prev_state == row.game_state:
                smoothed.append((row.game_state, 0))
            else:
                counter = Counter(buffer)
                current_state = counter.most_common()[0]
                smoothed.append(current_state)
            prev_state = smoothed[-1][0]

    df_framedets["game_state_smoothed"] = [s[0] for s in smoothed]


def add_state_transition(state_transitions, row: pd.Series, from_state: str, to_state: str):
    state_transitions.extend(
        [
            {
                "state": from_state,
                "frame_id": row.frame_id,
                "end_frame_id": row.frame_id,
                "end_sec": row.seconds,
                "end_time": row.time,
            },
            {
                "state": to_state,
                "frame_id": row.frame_id,
                "start_frame_id": row.frame_id,
                "end_frame_id": None,
                "start_sec": row.seconds,
                "end_sec": None,
                "start_time": row.time,
                "end_time": None,
            },
        ]
    )


def get_segments(df_framedets: pd.DataFrame, smoothing=False) -> List[Dict]:
    current_state = "out_of_game"
    state_transitions = []
    state_key = "game_state_smoothed" if smoothing else "game_state"
    for i, row in df_framedets.iterrows():
        if current_state != "in_game" and row[state_key] == "in_game":
            add_state_transition(state_transitions, row, "out_of_game", "in_game")
            current_state = "in_game"
        elif current_state == "in_game":
            if row[state_key] == "in_game":
                continue
            else:
                add_state_transition(state_transitions, row, "in_game", "out_of_game")
                current_state = row[state_key]
        elif current_state == "end_of_game" and row[state_key] in [
            "out_of_game",
            "end_of_game",
            "unknown",
        ]:
            current_state = "end_of_game"

    return state_transitions


def load_gt(filename: str = "seg_ground_truth_003.json"):
    """
    Loads ground truth for video segmentation from a json file that was exported from label-studio.
    """
    gt = {}
    anns = json.load(open(Path("/shared/gbiamby/geo/segment") / filename, "r"))
    for video in anns:
        segments, oog_segments = [], []
        video_id = (
            video["data"]["video_url"].replace("/data/local-files/?d=", "").replace(".mp4", "")
        )
        if not ("annotations" in video):
            print("no annotations key")
            continue
        for ann in video["annotations"][0]["result"]:
            segment = ann["value"]
            start_time, end_time = max(segment["start"], 0), max(0, segment["end"])
            if segment["labels"][0] == "in_game":
                segments.append((start_time, end_time))
            else:
                oog_segments.append((start_time, end_time))
        gt[video_id] = {
            "in_game_segs_count": len(segments),
            "oog_segs": oog_segments,
            "in_game_segs": segments,
        }
    return gt


def get_oog_segments(segments: List[Dict], game_state: str = None):
    i = 1
    segs = []
    while i + 1 < len(segments):
        seg = segments[i]
        seg.update(segments[i + 1])
        seg["duration_sec"] = seg["end_sec"] - seg["start_sec"]
        seg["duration_hms"] = datetime.utcfromtimestamp(seg["duration_sec"]).strftime("%H:%M:%S:%f")

        if game_state and seg["state"] == game_state:
            segs.append(seg)
        elif not game_state:
            segs.append(seg)
        i += 2
    return segs


def style_correct(s, props=""):
    return np.where(s == 1, props, "")


def style_wrong(s, props=""):
    return np.where(s == 0, props, "")


def compare_to_gt(segs: List[Dict], gt: Dict):
    gt_oog_segs = gt["oog_segs"]
    for seg in segs:
        if seg["state"] == "out_of_game":
            is_correct = (
                len(
                    list(
                        filter(
                            lambda s: float(s[1]) - 2.0 < seg["end_sec"] <= float(s[1]) + 2.0,
                            gt_oog_segs,
                        )
                    )
                )
                == 1
            )
            seg["is_correct"] = is_correct


def classify_frame(dets: pd.Series) -> str:
    """
    Input is a row of a pd.DataFrame. The row contains object detector output
    for the geoguessr UI elements.
    """
    label_set_base = set(dets["labels_set"])
    label_set = set(dets["labels_set"])
    if len(label_set) == 0:
        return "out_of_game"
    
    debug = False
    # if (
    #     label_set
    #     == {
    #         "btw_rounds_points_bar_blue_wide",
    #         "game_breakdown_blue",
    #         "play_again",
    #         "url",
    #     }
    #     or label_set == {"btw_rounds_points_bar_blue_wide", "game_breakdown_blue", "play_again"}
    # ):
    #     debug = True
    #     print("")
    #     print(f"boo label_set: {label_set}")

    for state, match_types in game_state_ui_elements.items():
        label_set = label_set_base

        if debug:
            print(f"state: {state}")

        if "do_not_match" in match_types:
            if len(label_set.intersection(match_types["do_not_match"][0])) > 0:
                if debug:
                    print(f"Skipping state {state} because of do_not_match")
                continue

        if "ignore" in match_types:
            label_set = label_set_base - match_types["ignore"][0]

        for ui_combo in match_types["exact"]:
            if ui_combo == label_set:
                if debug:
                    print("Found exact match: ", ui_combo)
                return state
            elif debug:
                print(f"No EXACT match for {label_set} vs {ui_combo}")

        if debug:
            print("label_set: ", label_set)
            print("label_set_base: ", label_set_base)
        for ui_combo in match_types["any"]:
            intersection = ui_combo.intersection(label_set)
            if len(ui_combo) == len(intersection):
                if debug:
                    print("Found ANY match type for ", ui_combo)
                return state
            elif debug:
                print(f"No ANY match for {label_set} vs {ui_combo}")

    if debug:
        print("Returning unknown for label_set: ", label_set)
    return "unknown"

In [ ]:
seg_gt = {
    "AF9uezxZDeE": {
        "in_game_segs_count": 5,
        "oog_segs": [(39, 141), (248, 275), (344, 352), (439, 440), (562, 650)],
    },
    "9RQUIk1OwAY": {
        "in_game_segs_count": 5,
        "oog_segs": [(185, 212), (394, 415), (597, 619), (801, 811), (994, 1022)],
    },
    "S5Ne5eoHxsY": {
        "in_game_segs_count": 5,
        "oog_segs": [(186, 212), (394, 405), (588, 608), (790, 809), (992, 1020)],
    },
    "nyHeQWnm8YA": {
        "in_game_segs_count": 5,
        "oog_segs": [(223, 231), (370, 377), (552, 554), (735, 742), (904, 929)],
    },
    "hZWt1PYH3hI": {
        "in_game_segs_count": 5,
        "oog_segs": [(117, 137), (314, 334), (471, 492), (660, 683), (866, 906)],
    },
    "dY1RXh-43q4": {
        "in_game_segs_count": 5,
        "oog_segs": [(158, 171), (287, 297), (418, 455), (577, 622), (740, 777)],
    },
    "83m9ys4kxro": {
        "in_game_segs_count": 5,
        "oog_segs": [(184, 206), (386, 398), (579, 592), (774, 806), (916, 951)],
    },
    "osTwgzWluVs": {
        "in_game_segs_count": 5,
        "oog_segs": [(147, 152), (272, 279), (360, 365), (525, 541), (671, 696)],
    },
    "o8qQAjkaXMM": {
        "in_game_segs_count": 20,
        "oog_segs": [
            (59, 67),
            (99, 107),
            (139, 147),
            (180, 195),
            (228, 262),
            (294, 299),
            (332, 346),
            (374, 377),
            (410, 425),
            (458, 482),
            (514, 528),
            (560, 566),
            (599, 607),
            (639, 647),
            (680, 708),
            (740, 757),
            (790, 793),
            (825.50, 838.50),
            (870.00, 886.75),
            # (825, 837),
            # (919, 937),
        ],
    },
}

In [ ]:
game_state_ui_elements = OrderedDict(
    {
        "pre_game": {
            "any": [
                ["play", "left_menu_dark"],
                ["play", "challenge_btn_orange"],
                ["game_about_to_start_box_white", "left_menu_dark"],
                ["battle_royale_start_menu_w_logo"],
                ["game_about_to_start_box_white", "next_orange_btn"],
                ["setup_round_time_limit_box"],
                ["battle_royale_start_menu_w_logo"],
                ["start_game"],
                ["invite_friends"],
                ["next_orange_btn"],
                ["start_challenge_orange"],
                ["start_game"],
            ],
            "exact": [],
        },
        "between_rounds": {
            "any": [
                ["left_menu_dark", "challenge_high_score_board"],
                ["play_next_round", "points_bar", "did_you_enjoy_this_location"],
                ["play_next_round", "points_bar_two_bars", "did_you_enjoy_this_location"],
                ["btw_rounds_points_bar_blue_wide", "play_next_round"],
                ["battle_royale_btw_rounds_green_box"],
                ["battle_royale_btw_rounds_red_box"],
            ],
            "exact": [
                ["in_game_mini_map", "status_bar", "points_bar", "game_title"],
                ["btw_rounds_points_bar_blue_wide"],
            ],
        },
        "between_round_or_game_ambiguous": {
            "any": [
                ["points_bar_two_bars", "did_you_enjoy_this_location", "status_bar"],
                ["between_rounds_box_white", "did_you_enjoy_this_location", "status_bar"],
                ["points_bar_two_bars", "status_bar"],
                ["points_bar_two_bars", "status_bar", "game_title"],
                ["points_bar_two_bars", "did_you_enjoy_this_location", "status_bar", "game_title"],
                ["points_bar_two_bars", "status_bar", "share_challenge_box_white"],
                ["points_bar_two_bars", "challenge_high_score_board", "status_bar"],
                ["points_bar_two_bars", "status_bar", "game_title", "share_challenge_box_white"],
                [
                    "between_rounds_box_white",
                    "did_you_enjoy_this_location",
                    "status_bar",
                    "game_title",
                ],
                ["points_bar", "left_menu_dark"],
                ["points_bar", "did_you_enjoy_this_location"],
                ["points_bar_two_bars", "did_you_enjoy_this_location"],
                ["status_bar", "points_bar", "game_title"],
                ["left_menu_dark"],
                ["left_menu_dark", "status_bar", "game_title"],
                ["left_menu_dark", "other"],
                ["left_menu_dark", "points_bar"],
                ["left_menu_dark", "points_bar_two_bars"],
            ],
            "exact": [],
            "do_not_match": [
                [
                    "in_game_mini_map",
                    "in_game_map_expanded",
                    "guess",
                    "guess_grey",
                    "make_a_guess",
                    "place_your_pin_grey",
                    "guess_w_icon_only",
                    "guess_expanded",
                    "guess_grey_expanded",
                    "make_a_guess_expanded",
                    "place_your_pin_grey_expanded",
                    "guess_w_icon_only_expanded",
                ],
            ],
        },
        "in_game": {
            "any": [
                # The ones with "in_game_mini_map" in first position will get auto-expanded:
                ["in_game_mini_map", "guess"],
                ["in_game_mini_map", "guess_grey"],
                ["in_game_mini_map", "make_a_guess"],
                ["in_game_mini_map", "place_your_pin_grey"],
                ["in_game_mini_map", "guess_w_icon_only"],
                # Won't get expanded:
                # ["status_bar", "in_game_mini_map"],
                # ["status_bar", "in_game_map_expanded"],
                ["status_bar_purple", "in_game_mini_map"],
                ["status_bar_purple", "in_game_map_expanded"],
                ["game_title", "in_game_map_expanded", "status_bar", "status_bar_white"],
                ["game_title", "in_game_mini_map", "status_bar"],
                ["game_title", "in_game_map_expanded", "status_bar"],
            ],
            "exact": [
                ["status_bar_white", "status_bar", "game_title"],
                ["status_bar_white", "other", "status_bar", "game_title"],
                ["game_title", "status_bar_white", "make_a_guess_expanded", "status_bar"],
                ["status_bar_purple"],
                ["game_title", "in_game_mini_map", "status_bar", "status_bar_white"],
                ["game_title", "in_game_map_expanded", "status_bar", "status_bar_white"],
            ],
            # Ignore these during matching elements in the 'any' and 'exacct' lists:
            "ignore": [
                [
                    "between_rounds_box_white",
                    "btw_rounds_points_bar_blue_wide",
                    "challenge_high_score_board",
                    "did_you_enjoy_this_location",
                    "high_score_box",
                    "leader_board",
                    "left_menu_dark",
                    "play_next_round",
                    "play",
                    "points_bar_two_bars",
                    "points_bar",
                    "show_full_results",
                    "show_high_score",
                    "try_another_map",
                    "battle_royale_start_menu_w_logo",
                ]
            ],
        },
        "between_games": {
            "any": [
                ["left_menu_dark", "challenge_high_score_board"],
                ["points_bar", "show_high_score"],
                ["points_bar_two_bars", "show_high_score"],
                ["try_another_map", "points_bar_two_bars", "show_full_results"],
                ["points_bar_two_bars", "show_full_results"],
                ["high_score_box", "leader_board"],
                ["battle_royale_game_over_dark_box"],
                ["battle_royale_knocked_out_end_game_red"],
                ["btw_rounds_points_bar_blue_wide", "game_breakdown_blue", "play_again"],
            ],
            "exact": [],
        },
        "out_of_game": {
            "any": [
                ["google_mini_map"],
            ],
            "exact": [
                [],
                ["other"],
            ],
        },
        "unknown": {
            "any": [],
            "exact": [],
        },
    }
)

for state, match_types in game_state_ui_elements.items():
    for match_type, ui_element_combos in match_types.items():
        print(match_type)
        if match_type == "any" and state == "in_game":
            # expand the map / guess button combos to include all possible
            # expanded/not-expanded combinations for the in_game_map and the corresponding
            # "guess" button
            map_combos = [
                ui_combo for ui_combo in ui_element_combos if ui_combo[0] == "in_game_mini_map"
            ]
            for ui_combo in map_combos:
                guess_button_label = ui_combo[1]
                ui_element_combos.append(["in_game_mini_map", f"{guess_button_label}_expanded"])
                ui_element_combos.append(["in_game_map_expanded", guess_button_label])
                ui_element_combos.append(["in_game_map_expanded", f"{guess_button_label}_expanded"])

        # Convert the ui element lists to sets:
        match_types[match_type] = [set(elements) for elements in ui_element_combos]
        if match_type == "ignore":
            print(match_types[match_type])

# game_state_ui_elements

In [ ]:
game_state_ui_elements

---

## Debug

In [ ]:
seg_gt_new = load_gt("seg_ground_truth_004.json")
print("num videos: ", len(seg_gt_new))
seg_gt_new.update(seg_gt)
print("num videos: ", len(seg_gt_new))

video_id = "AF9uezxZDeE"
video_id = "pMgqa0mOExo"
video_id = "dY1RXh-43q4"
video_id = "PzAXjKD4ZRg"
video_id = "osTwgzWluVs"
# gs_010_with_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-8b23604566/val/df_frame_dets-video_id_AF9uezxZDeE
df_framedets = load_detections(
    video_id,
    split="val",
    ## This one gets (5 / 9 correct, smoothing=True), (3 / 9 correct, smoothing=False):
    # model="output/gs_010_with_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-8b23604566",
    ### This one gets (6 / 9 correct, smoothing=True), ((6 / 9 correct, smoothing=False)):
    # model="output/gs_010_extra_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-024f52f6dd",
    ## This version looks promising. 15/18 correct on what is labelled of the new the new validation
    # (39 videos) so far:
    model="gs_011_extra_augs--geoscreens_011-model_faster_rcnn-bb_resnest50_fpn-3f36fb97fa",
)
df_framedets["game_state"] = df_framedets.apply(classify_frame, axis=1)
apply_smoothing(df_framedets, window_size=5, direction="forward")
seg = get_segments(df_framedets, smoothing=True)
segs_collapsed = get_oog_segments(seg)

#
print("video_id: ", video_id)
compare_to_gt(segs_collapsed, seg_gt_new[video_id])
df_seg = pd.DataFrame(segs_collapsed)
df_seg_styled = df_seg.style.apply(
    style_correct, props="color:white;background-color:green", axis=0, subset=["is_correct"]
).apply(style_wrong, props="color:white;background-color:red", axis=0, subset=["is_correct"])
display(df_seg_styled)
num_in_game_segments = df_seg[df_seg.state == "in_game"].shape[0]
print("num in_game: ", num_in_game_segments, ", gt: ", seg_gt_new[video_id])
if num_in_game_segments == seg_gt_new[video_id]["in_game_segs_count"]:
    print(colored("in_game segments count is: CORRECT!!!!!", color="green"))
else:
    print(colored("in_game segments count is: WRONG!!!", color="white", on_color="on_red"))

### Total Frames Per Game States

In [ ]:
print("Video_id: ", video_id)
pd.DataFrame(df_framedets.game_state.value_counts())

### Show ui combos that result in game_state "unknown":

In [ ]:
print("Video_id: ", video_id)
pd.DataFrame(
    df_framedets[df_framedets.game_state == "unknown"]
    .groupby(["labels_set"])
    .agg(cnt=("frame_id", "count"))
).sort_values("cnt", ascending=False)

### Show all Frames For game_state==unknown 

In [ ]:
print("Video_id: ", video_id)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df_framedets[df_framedets.game_state_smoothed == "unknown"][
            [
                "label_ids",
                "labels_set",
                # "scores",
                # "labels",
                # "bboxes",
                "label_set_count",
                "seconds",
                "time",
                "game_state",
                "game_state_smoothed",
            ]
        ]
    )

In [ ]:
# from geoscreens.pseudolabels import reverse_point

# for box in boxes:
#     print(
#         reverse_point(box["xmin"], box["ymin"], 1280, 720, 640),
#         reverse_point(box["xmax"], box["ymax"], 1280, 720, 640),
#     )

In [ ]:
print("Video_id: ", video_id)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        # df_framedets[(df_framedets.frame_id >= 3700)][
        # df_framedets[df_framedets.frame_id >= 0][
        # df_framedets[(2655	 <= df_framedets.frame_id) * (df_framedets.frame_id <= 3160)][
        df_framedets[df_framedets.labels_set == ("final_scores_box_beige",)][
        # df_framedets[
            [
                "frame_id",
                "label_ids",
                "labels_set",
                # "scores",
                "labels",
                "bboxes",
                "scores",
                "label_set_count",
                "seconds",
                "time",
                "game_state",
                "game_state_smoothed",
            ]
        ]
    )

### Find frames that have a specific set of UI elements

In [ ]:
print("Video_id: ", video_id)
df_framedets[df_framedets.labels_set == ("status_bar_white", "other", "status_bar", "game_title")]

---

## Segment many videos at once

In [ ]:
## This one gets (5 / 9 correct, smoothing=True), (3 / 9 correct, smoothing=False):
# model="output/gs_010_with_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-8b23604566",
## This one gets (7 / 9 correct, smoothing=True), ((7 / 9 correct, smoothing=False)):
# model="output/gs_010_extra_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-024f52f6dd",
model = "gs_011_extra_augs--geoscreens_011-model_faster_rcnn-bb_resnest50_fpn-3f36fb97fa"

In [ ]:
segments = {}
# val_ids = [
#     "AF9uezxZDeE",
#     "9RQUIk1OwAY",
#     "S5Ne5eoHxsY",
#     "nyHeQWnm8YA",
#     "hZWt1PYH3hI",
#     "dY1RXh-43q4",
#     "83m9ys4kxro",
#     "osTwgzWluVs",
#     "o8qQAjkaXMM",
# ]
seg_gt_new = load_gt("seg_ground_truth_004.json")
print("num videos: ", len(seg_gt_new))
seg_gt_new.update(seg_gt)
print("num videos: ", len(seg_gt_new))
val_ids = list(seg_gt_new.keys())
val_ids_no_detection_files = {
    val_id
    for val_id in val_ids
    if not Path(
        f"/shared/gbiamby/geo/segment/detections/{model}/val/df_frame_dets-video_id_{val_id}.csv"
    ).exists()
}
print("video_ids missing detection files: ", val_ids_no_detection_files)
val_ids = list(set(val_ids) - val_ids_no_detection_files)
print(f"Segmenting {len(val_ids)} videos...")
for video_id in val_ids:
    print("video_id: ", video_id)
    df_framedets = load_detections(video_id, split="val", model=model)
    df_framedets["game_state"] = df_framedets.apply(classify_frame, axis=1)
    apply_smoothing(df_framedets)
    seg = get_segments(df_framedets, smoothing=True)
    segments[video_id] = get_oog_segments(seg)

In [ ]:
for video_id, seg in segments.items():
    print("")
    print("")
    print("=" * 120)
    print("video_id: ", video_id)
    compare_to_gt(seg, seg_gt_new[video_id])
    df_seg = pd.DataFrame(seg)
    df_seg_styled = df_seg.style.apply(
        style_correct, props="color:white;background-color:green", axis=0, subset=["is_correct"]
    ).apply(style_wrong, props="color:white;background-color:red", axis=0, subset=["is_correct"])
    display(df_seg_styled)
    num_in_game_segments = df_seg[df_seg.state == "in_game"].shape[0]
    print(
        "num in_game: ", num_in_game_segments, ", gt: ", seg_gt_new[video_id]["in_game_segs_count"]
    )
    if num_in_game_segments == seg_gt_new[video_id]["in_game_segs_count"]:
        print(colored("in_game segments count is: CORRECT!!!!!", color="green"))
    else:
        print(colored("in_game segments count is: WRONG!!!", color="red", on_color="on_yellow"))
        print(seg_gt_new[video_id])

In [ ]:
for video_id, seg in segments.items():
    print("")
    print("video_id: ", video_id)
    # display(seg)
    for s in seg:
        print(s)

### Show the game states sequence:

In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df_framedets[df_framedets.seconds.isin([702.25, 702.5])][
            ["labels_set", "game_state", "time", "seconds"]
        ]
    )
    # display(df_framedets[["labels_set", "game_state", "time", "seconds"]])

In [ ]:
df_framedets.game_state.unique()

In [ ]:
current_state = "out_of_game"
state_transitions = []
for i, row in df_framedets.iterrows():
    if current_state != "in_game" and row.game_state == "in_game":
        state_transitions.extend(
            [
                {
                    "state": "out_of_game",
                    "end_frame_id": row.frame_id,
                    "end_frame_time": row.seconds,
                },
                {
                    "state": "in_game",
                    "start_frame_id": row.frame_id,
                    "start_frame_time": row.seconds,
                },
            ]
        )
        current_state = "in_game"
    elif current_state == "in_game":
        if row.game_state == "in_game":
            continue
        else:
            state_transitions.extend(
                [
                    {
                        "state": "in_game",
                        "end_frame_id": row.frame_id,
                        "end_frame_time": row.seconds,
                    },
                    {
                        "state": "out_of_game",
                        "start_frame_id": row.frame_id,
                        "start_frame_time": row.seconds,
                    },
                ]
            )
            current_state = row.game_state

state_transitions

## Scratch / Junk

---


In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df_framedets.loc[230:490][
            [
                "frame_id",
                "seconds",
                "labels",
                "labels_set",
                "game_state",
                "scores",
            ]
        ]
    )

In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_framedets.tail(100))

---